In [4]:

"""
This code is just an example, is not 100% optimized. 
Also, I'm not explaining why I use this preprocessing data because 
the aim of this project is testing Julia
But I think its really interesting for starting with Julia

# Example
```full_pipe("application_train.csv","application_test.csv") #Or path to those files
> 
```


Structure:
Imports and iniziate models.

Read dataset, separate features and targets
Drop columns
NUMERICAL DATA:
   - Use a simple "for" loop to impute missing values in numerical data using the median
   - disallowmissing! to change type from Union{missing,Float64} to Float64
   - coerce! to change the "scientific notation" of a column Count to a Continuous
        Basically, not all types of operations are allowed to be perform in a Count type
        (an ordinal sequence of number).
   - Use machine(Standar) to make the normalization
TEXT DATA:
   - Similar to the NUM_DATA, we impute missing data with the most_frecuent values
   - Then coerce! to change text data to a Multiclass
   - Finally, we use OneHotEncoding
BOOL_VALUES:
   - Just using a "for" loop to impute missing data with the most_frecuent values

Then we use the lightGBM to get the predictions







"""

using CSV #For reading CSV
using DataFrames 

include("constants.jl") #I import constants as modeule just to test
using .Constants

using StatsBase
using MLJ
using MLJ: fit! 
using MLJ: predict
using MLJ: transform#Dataframes and MLJ both has fit! and transform methods

using Missings #For dissallowing missing data

LGBMClassifier = @load LGBMClassifier verbosity=0 #Here we load the models that we're gonna use
OneHotEncoder = @load OneHotEncoder pkg=MLJModels verbosity=0

function full_pipe(train,test)


    lgb = LGBMClassifier() #initialised a model with default params

    train_df = DataFrame(CSV.File(train))
    test_df = DataFrame(CSV.File(test)) #Read .csv files

    target_train = train_df[!,["TARGET"]]
    target_test = test_df[!,["TARGET"]]
    select!(train_df, Not(["TARGET"]))
    select!(test_df, Not(["TARGET"]))
    select!(train_df, Not(Constants.COLUMNS_TO_DROP))
    select!(test_df, Not(Constants.COLUMNS_TO_DROP))

    # Last time I check it, MLJ didn't implemented a SimpleImputer. But is really easy to code
    num_features = Dict() #We want to store the median of numerical features
    

    for cols in Constants.COLUMNS_NUM

        num_features[cols] = median(skipmissing(train_df[:,cols]))
        
        replace!(train_df[!,cols] ,
            missing => num_features[cols])
        
        replace!(test_df[!,cols],  missing => num_features[cols])
    end
   
    train_df[!,Constants.COLUMNS_NUM] = disallowmissing!(train_df[!,Constants.COLUMNS_NUM])
    train_df[!,Constants.COLUMNS_NUM] = coerce!(train_df[!,Constants.COLUMNS_NUM], Count => Continuous)

    test_df[!,Constants.COLUMNS_NUM] = disallowmissing!(test_df[!,Constants.COLUMNS_NUM])
    test_df[!,Constants.COLUMNS_NUM] = coerce!(test_df[!,Constants.COLUMNS_NUM], Count => Continuous)

    stand1 = machine(Standardizer(),train_df[!,Constants.COLUMNS_NUM])

    fit!(stand1,verbosity=0)
    
    train_df[!,Constants.COLUMNS_NUM] = transform(stand1, train_df[!,Constants.COLUMNS_NUM])
    test_df[!,Constants.COLUMNS_NUM] = transform(stand1, test_df[!,Constants.COLUMNS_NUM])


    
    str_features = Dict()

    for cols in Constants.COLUMNS_STR

        str_features[cols] = StatsBase.mode(skipmissing(train_df[:,cols]))
        replace!(train_df[!,cols] ,
            missing => str_features[cols])

        replace!(test_df[!,cols],  missing => str_features[cols])
    end

    

    train_df[!,Constants.COLUMNS_STR] = disallowmissing!(train_df[!,Constants.COLUMNS_STR])
    train_df[!,Constants.COLUMNS_STR] = coerce!(train_df[!,Constants.COLUMNS_STR], Textual => Multiclass)

    test_df[!,Constants.COLUMNS_STR] = disallowmissing!(test_df[!,Constants.COLUMNS_STR])
    test_df[!,Constants.COLUMNS_STR] = coerce!(test_df[!,Constants.COLUMNS_STR], Textual => Multiclass)

    hot = machine(OneHotEncoder(drop_last=true), train_df[!,Constants.COLUMNS_STR])
    
    fit!(hot,verbosity=0) 
    train_df = hcat(select(train_df, Not(Constants.COLUMNS_STR)),
                    transform(hot, train_df[!,Constants.COLUMNS_STR]))

    test_df = hcat(select(test_df, Not(Constants.COLUMNS_STR)),
                    transform(hot, test_df[!,Constants.COLUMNS_STR]))

    bool_features = Dict()

    for cols in Constants.COLUMNS_BOOL
        
        bool_features[cols] = StatsBase.mode(skipmissing(train_df[:,cols]))
        replace!(train_df[!,cols] ,
            missing => bool_features[cols])

        replace!(test_df[!,cols], missing => bool_features[cols])
    end

    target_train = coerce!(target_train,  Count => Multiclass )
    target_test = coerce!(target_test,  Count => Multiclass ) 

    lgbm = machine(lgb, train_df, target_train[!,1])
    
    fit!(lgbm)
    
    train_predict = predict(lgbm, train_df)
    test_predict = predict(lgbm, test_df)

    train_auc = auc(train_predict, target_train[!,1])
    test_auc = auc(test_predict, target_test[!,1])
    println(train_auc )
    println(test_auc )
    return train_auc, test_auc
    
end




full_pipe (generic function with 1 method)

In [53]:
using BenchmarkTools
@btime full_pipe("application_train.csv","application_test.csv")

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc LightGBM.LGBMClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Union{Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}, Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}, Abstrac

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.067905 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6227
[LightGBM] [Info] Number of data points in the train set: 215257, number of used features: 184
[LightGBM] [Info] Start training from score -0.084162
[LightGBM] [Info] Start training from score -2.516800
0.735051420150909
0.7321127469927498


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc LightGBM.LGBMClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Union{Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}, Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}, Abstrac

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.053715 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6227
[LightGBM] [Info] Number of data points in the train set: 215257, number of used features: 184
[LightGBM] [Info] Start training from score -0.084162
[LightGBM] [Info] Start training from score -2.516800
0.735051420150909
0.7321127469927498


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc LightGBM.LGBMClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Union{Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}, Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}, Abstrac

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.048936 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6227
[LightGBM] [Info] Number of data points in the train set: 215257, number of used features: 184
[LightGBM] [Info] Start training from score -0.084162
[LightGBM] [Info] Start training from score -2.516800
0.735051420150909
0.7321127469927498


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc LightGBM.LGBMClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}, AbstractVector{Multiclass{2}}}
│ 
│ fit_data_scitype(model) = Union{Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}, Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}, Abstrac

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.104135 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6227
[LightGBM] [Info] Number of data points in the train set: 215257, number of used features: 184
[LightGBM] [Info] Start training from score -0.084162
[LightGBM] [Info] Start training from score -2.516800
0.735051420150909
0.7321127469927498
  15.232 s (37537746 allocations: 4.66 GiB)


(0.735051420150909, 0.7321127469927498)